In [1]:
# This adds the path to import the development version (git repo) of NDI Python
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from ndi import NDI_Object, Document, Experiment, Epoch, Probe, Channel
from ndi import DaqSystem, FileNavigator
from ndi.database.file_system import FileSystem as Database, BinaryCollection
from ndi.query import Query as Q, AndQuery, OrQuery
from ndi.database.utils import print_everything_in, destroy_everything_in

In [3]:
from ndi.daqreaders import MockReader

In [4]:
# Connecting to FileSystem database, passing in a path to a directory
db = Database('../test_db')
destroy_everything_in(db)

In [5]:
# Creating/connecting to binary collection, passing in path and collection name
bc = BinaryCollection('../test_db', name='demo')

In [6]:
# Creating FileNavigator object, passing in regex patterns
fn = FileNavigator(epoch_file_patterns=['.*\\.wav', '.*\\.txt'], 
                   metadata_file_pattern='.*\\.txt')

# Mock EpochProbeMap class used just for this demo
class MockEpochProbeMap: pass

# Creating DaqSystem objects
daqs = [
    DaqSystem('ds0', fn, MockReader, MockEpochProbeMap),
    DaqSystem('ds1', fn, MockReader, MockEpochProbeMap)
]

In [7]:
# Creating an Experiment object and connecting it to its context
exp = Experiment('test_experiment').connect(
    directory='../tests/data/intracell_example',
    database=db,
    binary_collection=bc,
    daq_systems=daqs,
    load_existing=False
)

In [8]:
# Creating and adding probes to the database through the experiment
probes = [
    Probe('p0', 0, 'ntrode', daq_system_id=daqs[0].id),
    Probe('p1', 1, 'ntrode', daq_system_id=daqs[0].id),
    Probe('p2', 2, 'ntrode', daq_system_id=daqs[0].id),
]

for p in probes:
    exp.add_probe(p)

In [9]:
# Showing that probes were added
exp.get_probes()

In [10]:
# Creating and adding channels to the database through the probes
channels0 = [
    Channel('p0c0', 0, 'time', 'file0'),
    Channel('p0c1', 1, 'digital_out', 'file0'),
    Channel('p0c2', 2, 'mark', 'file0'),
]
channels1 = [
    Channel('p1c0', 0, 'time', 'file1'),
    Channel('p1c1', 1, 'digital_out', 'file1'),
    Channel('p1c2', 2, 'mark', 'file1'),
]
for c in channels0:
    probes[0].add_channel(c)
for c in channels1:
    probes[1].add_channel(c)

In [11]:
# Getting channels via probes
probes[0].get_channels()

In [12]:
# Getting channels via experiment
exp.get_channels()

No DAQ reader set to <ndi.core.Channel object at 0x106c0c820>.
No DAQ reader set to <ndi.core.Channel object at 0x106c0cb20>.
No DAQ reader set to <ndi.core.Channel object at 0x106c0c940>.
No DAQ reader set to <ndi.core.Channel object at 0x112dd9340>.
No DAQ reader set to <ndi.core.Channel object at 0x11fb282e0>.
No DAQ reader set to <ndi.core.Channel object at 0x11fb28fd0>.


In [13]:
# Creating and adding epochs to the database through experiment
epochs = [
    Epoch(daq_system_ids=[daqs[0].id]),
    Epoch(daq_system_ids=[daqs[0].id])
]
for e in epochs:
    exp.add_epoch(e)

In [14]:
# Getting epochs via experiment
exp.get_epochs()

[<ndi.core.Epoch at 0x11fb45610>, <ndi.core.Epoch at 0x11fb45ca0>]

In [15]:
# Creating and adding a document to the database though experiment
doc = Document({'hello': 'world'}, name='doc', type_='independent')
exp.add_document(doc)

In [16]:
# Finding the just-added document
exp.ctx.db.find_by_id(doc.id).data

{'_metadata': {'name': 'doc',
  'type': 'independent',
  'experiment_id': '7bedd384549348b5ab466af8b59580aa',
  'parent_id': '',
  'asc_path': '',
  'version_depth': 0,
  'latest_version': True},
 '_dependencies': {},
 '_depends_on': ['7bedd384549348b5ab466af8b59580aa'],
 'hello': 'world'}

In [17]:
# Getting document dependencies
exp.current.get_document_dependencies()

{'doc': <ndi.document.Document at 0x11fb5f370>}

In [18]:
# Deleting the document
print(doc.id)
doc.delete(force=True)

641d26104c064499871b5cf8b1254f8a


In [19]:
# accessing exp with current ensures that the experiment is up to date with the database
# is equivalent to ndi_object.refresh()
exp.current.get_document_dependencies()

{}

In [20]:
doc.data

'This object has been deleted.'

In [21]:
# Adding more complex document to database
doc = Document({"list": ["abc", "def", "ghi"]}, 'list-doc')
db.add(doc)

# Finding document using query
[d.data for d in db.find(Q('list').contains('def'))]

[{'_metadata': {'name': 'list-doc',
   'type': '',
   'experiment_id': '',
   'parent_id': '',
   'asc_path': '',
   'version_depth': 0,
   'latest_version': True},
  '_dependencies': {},
  '_depends_on': [],
  'list': ['abc', 'def', 'ghi']}]

In [22]:
# Reconnect to an experiment that already exists in database
Experiment('test_experiment').connect(
    database=db,
    binary_collection=bc,
    load_existing=True
).document.data

{'_metadata': {'name': 'test_experiment',
  'type': 'ndi_experiment',
  'experiment_id': '7bedd384549348b5ab466af8b59580aa',
  'parent_id': '',
  'asc_path': '',
  'version_depth': 0,
  'latest_version': True},
 '_dependencies': {},
 '_depends_on': []}

In [23]:
# Create new experiment that doesn't exist in the database
Experiment('test_dfjslfjdkf').connect(
    database=db,
    binary_collection=bc,
    load_existing=False
).document.data

{'_metadata': {'name': 'test_dfjslfjdkf',
  'type': 'ndi_experiment',
  'experiment_id': '6c4d0be6c95c4aa58890501a8ed8c375',
  'parent_id': '',
  'asc_path': '',
  'version_depth': 0,
  'latest_version': True},
 '_dependencies': {},
 '_depends_on': []}